# Out of Band Protocol - Receiver

The out of band protocol allows agents to exchange messages without requiring a DIDComm channel. This can be used to establish a connection, request a presentation or issue a credential. 

The RFC is described [here](https://github.com/hyperledger/aries-rfcs/tree/master/features/0434-outofband)

Begin in the [sender notebook](http://localhost:8888/notebooks/Part%208%20-%20Out%20of%20Band%20Protocol.ipynb).

## 5. Initialise the Controller

In [1]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

IPython autoawait is `on`, and set to use `asyncio`


## 6. Configure Connection Listeners

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    print("Connection Webhook : ", payload)
    
connections_listener = {
    "topic": "connections",
    "handler": connections_handler
}


agent_controller.register_listeners([connections_listener], defaults=True)

## 7. Receive Out of Band Invitation

**You should have copied this from the Sender notebook (Step 3)**

In [3]:
import json
oob_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': 'e536ed97-2e32-4133-bf20-7ce998739267', 'label': 'Alice', 'handshake_protocols': ['https://didcomm.org/connections/1.0/invitation', 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation'], 'request~attach': [], 'service': [{'id': '#inline', 'type': 'did-communication', 'recipientKeys': ['did:key:z6Mkn6wscbEJxHZ4Fnf31DwuSfADcWqh6Ncc3fpNJTYHYZHP'], 'routingKeys': [], 'serviceEndpoint': 'http://172.17.0.1:8020'}]}
response = await agent_controller.oob.receive_invitation(oob_invite)
print(response)

Connection Webhook :  {'connection_id': '4f47cb7e-be35-4a34-8c4c-f0d987aec3aa', 'created_at': '2020-10-30 12:50:49.322105Z', 'routing_state': 'none', 'invitation_key': '8egq2Lysck4b9HpLKez4bZcDnwZqgVNFMeuSUBaGdLW1', 'state': 'invitation', 'accept': 'auto', 'initiator': 'external', 'invitation_mode': 'once', 'updated_at': '2020-10-30 12:50:49.322105Z', 'their_label': 'Alice'}
Connection Webhook :  {'connection_id': '4f47cb7e-be35-4a34-8c4c-f0d987aec3aa', 'created_at': '2020-10-30 12:50:49.322105Z', 'routing_state': 'none', 'invitation_key': '8egq2Lysck4b9HpLKez4bZcDnwZqgVNFMeuSUBaGdLW1', 'state': 'request', 'accept': 'auto', 'initiator': 'external', 'invitation_mode': 'once', 'request_id': 'db6d03a3-04ca-4ed3-9da6-36fc31f3ad95', 'updated_at': '2020-10-30 12:50:49.333258Z', 'my_did': '3oyc4duyVFbKq6wKLhqfv5', 'their_label': 'Alice'}
{'connection_id': '4f47cb7e-be35-4a34-8c4c-f0d987aec3aa', 'created_at': '2020-10-30 12:50:49.322105Z', 'routing_state': 'none', 'invitation_key': '8egq2Lysck

## 8. Get Connection

In [4]:
connection_id = response["connection_id"]
print(connection_id)

connection = await agent_controller.connections.get_connection(connection_id)

print(connection)
print("Connection State : ", connection["state"])

4f47cb7e-be35-4a34-8c4c-f0d987aec3aa
{'connection_id': '4f47cb7e-be35-4a34-8c4c-f0d987aec3aa', 'created_at': '2020-10-30 12:50:49.322105Z', 'their_did': '6RoJMQnJikyY2Em2pz7p8Q', 'routing_state': 'none', 'invitation_key': '8egq2Lysck4b9HpLKez4bZcDnwZqgVNFMeuSUBaGdLW1', 'state': 'response', 'accept': 'auto', 'initiator': 'external', 'invitation_mode': 'once', 'request_id': 'db6d03a3-04ca-4ed3-9da6-36fc31f3ad95', 'updated_at': '2020-10-30 12:50:49.421094Z', 'my_did': '3oyc4duyVFbKq6wKLhqfv5', 'their_label': 'Alice'}
Connection State :  response


## 9. Send Trust Ping

This moves the connection to the active state

In [6]:
await agent_controller.messaging.trust_ping(connection_id, "active")

{'thread_id': '85e649b0-0fea-4691-a5ae-61dfc51e643c'}

Connection Webhook :  {'connection_id': '4f47cb7e-be35-4a34-8c4c-f0d987aec3aa', 'created_at': '2020-10-30 12:50:49.322105Z', 'their_did': '6RoJMQnJikyY2Em2pz7p8Q', 'routing_state': 'none', 'invitation_key': '8egq2Lysck4b9HpLKez4bZcDnwZqgVNFMeuSUBaGdLW1', 'state': 'active', 'accept': 'auto', 'initiator': 'external', 'invitation_mode': 'once', 'request_id': 'db6d03a3-04ca-4ed3-9da6-36fc31f3ad95', 'updated_at': '2020-10-30 12:52:25.954656Z', 'my_did': '3oyc4duyVFbKq6wKLhqfv5', 'their_label': 'Alice'}


## End Tutorial


In [7]:
await agent_controller.terminate()